In [1]:
import os
import torch
import pandas as pd

from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset, DatasetDict
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/home/uj-user/Yo/hybrid-ltm/ltm-venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = "/home/uj-user/Yo/hybrid-ltm/data_eval/context-fms/MS_DG/test.jsonl"
data_path = "/home/uj-user/Yo/hybrid-ltm/data_eval/context-fms/MS_DG/test_last_session.jsonl"
# model_name = "facebook/blenderbot-3B"
model_name = "facebook/blenderbot-400M-distill"
# model_name = "gpt2"
MODEL_MAX_LEN = 64
TARGET_LEN = 16
device = "cuda:2" if torch.cuda.is_available() else "cpu"

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [4]:
dataset = pd.read_json(data_path, lines=True)
dataset

,input
0,```\nDialogue Session #3:\nDoctor:Hello Patien...
1,```\nDialogue Session #3:\nNeighbors A:Hey the...
2,"```\nDialogue Session #4:\nCo-workers A:Hey, B..."
3,"```\nDialogue Session #2:\nMentee:Mentor, I ha..."
4,```\nDialogue Session #5:\nClassmates A:Sorry ...
...,...
75,```\nDialogue Session #4:\nHusband:You know wh...
76,"```\nDialogue Session #4:\nClassmates B:Hey, C..."
77,```\nDialogue Session #3:\nClassmates A:I've b...
78,```\nDialogue Session #2:\nHusband:I found a s...


In [5]:
import re
dataset['prep_input'] = dataset['input'].map(lambda x: '```'+ '```'.join(x.split('```')[1:-1]) +'```')
text_list = dataset['prep_input'][1].split('Dialogue Session #')[1:] # dialogue session remove
result_list = [re.sub(r'^\d+:\s*', '', line) for line in text_list] # #1 숫자 제거
result_list = ['\n'.join(line.split(':', 1)[1].strip() for line in lines.split('\n') if ':' in line) for lines in result_list]
result_list = [line.replace('\n', ' ').replace('```', '').strip() for line in result_list] # 불필요 개행문자, 공백제거
result_list = ' '.join(result_list)
result_list = result_list[:-3].strip()
result_list

'Hey there, have you ever considered pursuing journalism as a career? Journalism? No, not really. Why do you ask?'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation_side='left', use_fast=False)
# tokenizer.add_special_tokens({'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'})
tokenizer

BlenderbotTokenizer(name_or_path='facebook/blenderbot-400M-distill', vocab_size=8008, model_max_length=128, is_fast=False, padding_side='right', truncation_side='left', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	8008: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

In [14]:
ans_jsons = []
for i, line in enumerate(tqdm(dataset['input'])):
    
    dataset['prep_input'] = dataset['input'].map(lambda x: '```'+ '```'.join(x.split('```')[1:-1]) +'```')
    text_list = dataset['prep_input'][i].split('Dialogue Session #')[1:] # dialogue session remove
    result_list = [re.sub(r'^\d+:\s*', '', line) for line in text_list] # #1 숫자 제거
    result_list = ['\n'.join(line.split(':', 1)[1].strip() for line in lines.split('\n') if ':' in line) for lines in result_list]
    result_list = [line.replace('\n', ' ').replace('```', '').strip() for line in result_list] # 불필요 개행문자, 공백제거
    result_list = ' '.join(result_list)
    result_list = result_list[:-3].strip()
    # print(result_list)

    input = tokenizer(result_list, max_length=MODEL_MAX_LEN-TARGET_LEN, truncation=True, add_special_tokens = False, return_tensors='pt').to(device)#.input_ids
    output_ids = model.generate(
        **input,
        # do_sample=True,
        # repetition_penalty=1.1,
        # no_repeat_ngram_size=2,
        # # temperature=0.2,
        # top_p=0.8,
        # max_length=MODEL_MAX_LEN,
        encoder_no_repeat_ngram_size= None
    )
    output_ids = output_ids[0][len(input.input_ids[0]):]
    outputs = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    print('input:\n', tokenizer.decode(input.input_ids[0]))
    print('output:\n', outputs)
    if i%20==10:
        break

    ans_jsons.append(
        {
            "question_id": i,
            "text": outputs,
            "model_id": model_name,
            "metadata": {},
        }
    )

  2%|▎         | 2/80 [00:00<00:11,  7.07it/s]

input:
 , a few years ago I went to Europe. That's great. Did you have any issues with your health during that trip? No, everything was fine. Although, I did experience some jet lag when I arrived back home.
output:
 
input:
  Hey there, have you ever considered pursuing journalism as a career? Journalism? No, not really. Why do you ask?
output:
 


  6%|▋         | 5/80 [00:00<00:08,  9.31it/s]

input:
  that, A. It's never easy losing a loved one. No, it's not. But I learned a lot about appreciating the people in my life while I had them, and that's something I try to remember every day.
output:
 
input:
 , Mentee. I'm glad you were able to offer some words of encouragement to personY. It's important to remember that everyone goes through tough times, but it's how we handle those tough times that defines us.
output:
 
input:
  what happened? It's a complicated story, but I can share it with you sometime. For now, let's just enjoy our time together. Remember the time I helped you with your math homework? That was a good memory.
output:
 


 10%|█         | 8/80 [00:00<00:06, 10.40it/s]

input:
  off of. Yeah, you're right. Thanks for your help, as always. Anytime, happy to be of assistance. Speaking of problem-solving, did you ever get that leak in your kitchen fixed?
output:
 
input:
  remember. By the way, did you hear about the homeless shelter that opened up downtown recently? No, I hadn't heard about that. That's great news though. It's always important to have resources for those in need.
output:
 
input:
  town. The main character is torn between her love for the town and her desire to pursue her dreams in the city. That sounds really interesting. I'm looking forward to reading it. Have you had any other writing projects recently?
output:
 


 12%|█▎        | 10/80 [00:01<00:06, 10.74it/s]

input:
 . That's great! Have you guys talked since then? Yeah, we've been talking and I'm really excited to see where it goes. I'm happy for you. Just make sure you're honest with her and treat her right.
output:
 
input:
  skills by reading a variety of books. Remember when we talked about that a few days after I won that challenge? Of course, I do. That's great to hear. Did you learn anything new from the books you've read recently?
output:
 


 12%|█▎        | 10/80 [00:01<00:08,  8.66it/s]

input:
  sorry, I didn't realize it was bothering you. It's just that I want to spend time with you too. Maybe we could read together or something? That's a good idea. We could start a book club or something.
output:
 


In [8]:
filename = f"predictions/{model_name}_{data_path.split('/')[-2]}.json"
save_path = os.path.dirname(filename)
os.makedirs(save_path, exist_ok=True)
df_ans_json = pd.DataFrame(ans_jsons)
df_ans_json.to_json(filename, orient='records', lines=True)

In [9]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

# mname = "facebook/blenderbot-400M-distill"
# model = BlenderbotForConditionalGeneration.from_pretrained(mname)
# tokenizer = BlenderbotTokenizer.from_pretrained(mname)

In [21]:
UTTERANCE = "Did you learn anything new from the books you've read recently?"
# That's unfortunate. Are they trying to lose weight or are they just trying to be healthier? They are trying to get healthier, but it's hard for them to stick to it."
inputs = tokenizer([UTTERANCE],add_special_tokens= False, return_tensors="pt").to(device)
reply_ids = model.generate(**inputs, encoder_no_repeat_ngram_size= None)
print(tokenizer.batch_decode(reply_ids))

[" Did you learn anything new from the books you've read recently? I'm with you on that one.</s>"]
